In [5]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
import numpy as np

### First Attempt just using Beautiful Soup

In [6]:
levels = ['college', 'high-school']
sports = ['football', 'basketball', 'baseball', 'womens-basketball', 'volleyball', 'gymnastics',
         'mens-lacrosse', 'womens-lacrosse', 'mens-soccer', 'womens-soccer', 'softball', 'womens-track',
         'mens-golf', 'womens-golf', 'mens-hockey', 'womens-hockey', 'mens-swimming', 'womens-swimming']

all_names = []

for level in levels:
    for sport in sports:

        URL = f'https://www.on3.com/nil/rankings/player/{level}/{sport}/'

        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        results = soup.find(class_="NilRankingsPageComponent_nilRankingsList__t14Ao")

        try:
            names_sport = results.find_all('a', class_="MuiTypography-root MuiLink-root MuiLink-underlineNone MuiTypography-h5 MuiTypography-colorPrimary")
            for name in names_sport:
                all_names.append(name)
        except:
            # print("Whoops! No athletes appear to have NIL deal for {} {}".format(level, sport))
            continue
    #end
#end

In [7]:
print(len(all_names))

414


It looks like there were only 414 athletes using this section of the On3 database... This is such a small number of athletes that we should investiage looking at different sections of the website

### Using Selenium & Beatuiful Soup

In [7]:
sports = ['basketball']
#years = ['2022','2023','2024','2025','2026']
years = ['2023']

ages = []
athlete_grade = []

for year in years:
    for sport in sports:
        
        # Selenium Driver to click on "Load More"
        driver = webdriver.Chrome()
        URL = f'https://www.on3.com/db/rankings/industry-player/{sport}/{year}/'
        driver.get(URL)
        
        # while True:
        #     try:
        #         load_more_button = WebDriverWait(driver, 10).until(
        #             EC.element_to_be_clickable((By.XPATH, "//span[@class='MuiButton-label' and contains(text(), 'Load More')]"))
        #         )
        #         load_more_button.click()
        #         time.sleep(10)
        #     except:
        #         break
        
        # Beautiful Soup - for each player
        page_source = driver.page_source
        page_soup = BeautifulSoup(page_source, 'html.parser')
        results = page_soup.find(class_="PlayerRankings_playerRankings__jvfFg")

        athletes = results.find_all('a', class_="MuiTypography-root MuiLink-root MuiLink-underlineHover MuiTypography-h5 MuiTypography-colorPrimary")
        # It is possible that two athletes have the same name: John Smith
        # We cannot use a dictionary so a list of tuples is used for now
        names = [athlete.text for athlete in athletes]

        # Generate a list of links that we can iterate through
        links = [athlete['href'] for athlete in athletes]
        base_url = "https://www.on3.com/"
        athlete_links = [urljoin(base_url, link) for link in links]

        for athlete_link in athlete_links:
            # We can now go into each individual athletes page
            # for that sport, for that year, after loading more
            driver.get(athlete_link)
            time.sleep(3)

            athlete_source = driver.page_source
            page_soup = BeautifulSoup(athlete_source, 'html.parser')

            if page_soup.find(class_="MuiTypography-root CollegeRanking_span__qtAfW MuiTypography-subtitle1 MuiTypography-colorPrimary") is not None:
                age = page_soup.find(class_="MuiTypography-root CollegeRanking_span__qtAfW MuiTypography-subtitle1 MuiTypography-colorPrimary")
                ages.append(age.text)
            else:
                ages.append(np.nan)
            
            # The "EXP" class_name folows the exact same class_name as "Year"...
            if page_soup.find(class_="MuiTypography-root CollegeRanking_span__qtAfW MuiTypography-subtitle1 MuiTypography-colorTextPrimary") is not None:
                year_ = page_soup.find(class_="MuiTypography-root CollegeRanking_span__qtAfW MuiTypography-subtitle1 MuiTypography-colorTextPrimary")
                athlete_grade.append(year_.text)
            else:
                athlete_grade.append(np.nan)
        #end
        print(len(names), len(ages), len(athlete_grade))
        print(names[0], ages[0], athlete_grade[0])
    #end
#end

        # This successfully takes us back to the previous page, but it might require us to reload every athlete...
        # driver.back()
        # url = driver.current_url
        # print(url)

driver.quit()

50 50 50
Isaiah Collier 18 2023 - present


In [21]:
print(len(names))
print(names[0])

50
Isaiah Collier


In [12]:
# sports = ['basketball']
# #years = ['2022','2023','2024','2025','2026']
# years = ['2023']


# urls = {}

# for year in years:
#     for sport in sports:

        
            
        # page = requests.get(URL)
        # soup = BeautifulSoup(page.content, 'html.parser')
        #results = soup.find(class_="PlayerRankings_playerRankings__jvfFg")
        
        # page_soup = soup(driver.page_source, 'html.parser')
        # print(page_soup)
        # results = page_soup.find(class_="PlayerRankings_playerRankings__jvfFg")
        # try:
        #     names_sport = results.find_all('a', class_="MuiTypography-root MuiLink-root MuiLink-underlineHover MuiTypography-h5 MuiTypography-colorPrimary")
        #     for name in names_sport:
        #         urls[name.text] = name['href']
        # except:
        #     print("Whoops! No athletes appear to have NIL deal for {} {}".format(level, sport))
        #     continue
    #end
#end

In [13]:
# all_info = {}

# for name, url in urls.items():
#     all_info[name] = []
#     URL = f'https://www.on3.com{url}/recruiting'
#     page = requests.get(URL)
#     soup = BeautifulSoup(page.content, 'html.parser')
    
    
    
#     results = soup.find(class_="PlayerRecruiting_playergrid__wYlb_")
#     player_info = results.find_all('span', class_="MuiTypography-root MeasurementInfo_text__dCryI MuiTypography-body1 MuiTypography-colorTextPrimary")
#     for info in player_info:
#         all_info[name].append(info.text)
    
#     results = soup.find(class_="Rankings_container__U2afk")
#     rankings_info = results.find_all('p', class_="MuiTypography-root Rankings_industryRating__9uavm MuiTypography-body1 MuiTypography-colorTextPrimary")
#     for info in rankings_info:
#         all_info[name].append(info.text)
    
    
#     results = soup.find(class_="PlayerInterestsModule_targets__rxz4U")
#     recruitment_info = results.find_all(['a','h6'], class_=["MuiTypography-root MuiLink-root MuiLink-underlineNone PlayerInterestsItem_teamName__FeBHv MuiTypography-h5 MuiTypography-colorPrimary","MuiTypography-root PlayerInterestsItem_predictionLabel__mT201 MuiTypography-subtitle1 MuiTypography-colorTextPrimary"])
#     for info in  recruitment_info:
#         all_info[name].append(info.text)
